In [18]:
import polars as pl
import glob, os
import json

In [2]:
# ===================== CONSTANT =========================
ALL_TIMELINE_JSON_PATH = "../res/data/timelines/*.json"

In [21]:

# 1) Read all timeline JSON files (one file per match)
paths = glob.glob(ALL_TIMELINE_JSON_PATH)

dfs = []
for p in paths:
    df = pl.read_json(p).with_columns(
        pl.col("metadata").struct.json_encode().alias("metadata_json"),
        pl.col("info").struct.json_encode().alias("info_json"),
        pl.lit(os.path.basename(p)).alias("source_file"),
    ).select("metadata_json", "info_json", "source_file")
    dfs.append(df)

timeline_matches = pl.concat(dfs, how="diagonal", rechunk=True)



In [22]:
# timeline_matches: 有 metadata_json / info_json / source_file 三列
rows = []
for m_json, i_json, src in timeline_matches.select(
        "metadata_json", "info_json", "source_file"
    ).iter_rows():
    m = json.loads(m_json)
    i = json.loads(i_json)
    rows.append({
        "match_id":       m.get("matchId"),
        "puuid_list":     m.get("participants"),
        "end_result":     i.get("endOfGameResult"),
        "frame_interval": i.get("frameInterval"),
        "frames":         i.get("frames"),
        "source_file":    src,
    })

decoded = pl.DataFrame(rows)


In [24]:
# 2) frames（每行=一场的一分钟）
frames = (
    decoded
    .explode("frames")
    .drop_nulls("frames")
    .unnest("frames")
    .with_columns((pl.col("timestamp")/60000).cast(pl.Int64).alias("minute"))
)

In [26]:
# 3) 事件表（避免 timestamp 重名）
events_all = (
    frames
    .select(
        "match_id",
        pl.col("timestamp").alias("frame_ts"),  # 帧级时间
        "minute",
        "events",
    )
    .explode("events")
    .drop_nulls("events")
    .unnest("events")                           # 这里会产生事件自己的 timestamp
    .rename({"timestamp": "event_ts"})          # 事件级时间改名
    .with_columns([
        (pl.col("frame_ts")/60000).cast(pl.Int64).alias("frame_minute"),
        # 有些事件可能没有 event_ts，用帧 minute 兜底
        pl.when(pl.col("event_ts").is_not_null())
          .then((pl.col("event_ts")/60000).cast(pl.Int64))
          .otherwise(pl.col("minute"))
          .alias("event_minute"),
    ])
)


In [27]:
# 4) participant_id <-> puuid
pid_map = (
    decoded
    .select("match_id", "puuid_list")
    .explode("puuid_list")
    .with_row_index("participant_id", offset=1)
    .rename({"puuid_list": "puuid"})
)

In [30]:
# 5) participant frames（无 melt 版本）
pf_wide = (
    frames
    .select("match_id", "timestamp", "minute", "participantFrames")
    .drop_nulls("participantFrames")
    .unnest("participantFrames")   # -> 列名 "1","2",... 每列一个 struct
)

# 动态找出所有参赛者列（列名是纯数字的）
pf_cols = [c for c in pf_wide.columns if c.isdigit()]
pid_vals = [int(c) for c in pf_cols]

pf_all = (
    pf_wide
    .with_columns([
        pl.concat_list([pl.col(c) for c in pf_cols]).alias("pf_list"),     # list<struct>
        pl.lit(pid_vals).alias("pid_list"),                                # list<int>
    ])
    .select("match_id", "timestamp", "minute", "pf_list", "pid_list")
    .explode(["pf_list", "pid_list"])                                      # 每个元素一行
    .rename({"pf_list": "pf", "pid_list": "participant_id"})
    .unnest("pf")                                                          # 展开每位选手该分钟的结构
    # 位置坐标展开（可能缺失）
    .with_columns([
        pl.when(pl.col("position").is_not_null()).then(pl.col("position").struct.field("x")).otherwise(None).alias("pos_x"),
        pl.when(pl.col("position").is_not_null()).then(pl.col("position").struct.field("y")).otherwise(None).alias("pos_y"),
    ])
    .drop("position")
    # championStats / damageStats 展开（可能缺失）
    .with_columns([
        pl.when(pl.col("championStats").is_not_null()).then(pl.col("championStats")).otherwise(None).alias("championStats"),
        pl.when(pl.col("damageStats").is_not_null()).then(pl.col("damageStats")).otherwise(None).alias("damageStats"),
    ])
    .unnest("championStats")
    .unnest("damageStats")
    .join(pid_map, on=["match_id", "participant_id"], how="left")
)


In [31]:
frames.select("match_id","timestamp","minute").write_parquet("frames.parquet")
events_all.write_parquet("events.parquet")
pf_all.write_parquet("participant_frames.parquet")


In [34]:
# 每分钟增量
pf_deltas = (
    pf_all
    .sort(["match_id","participant_id","minute"])
    .group_by(["match_id","participant_id","puuid"])
    .agg([
        pl.col("minute"),
        pl.col("minionsKilled").diff().alias("cs_delta"),
        pl.col("totalGold").diff().alias("gold_delta"),
        pl.col("xp").diff().alias("xp_delta"),
    ])
    .explode(["minute","cs_delta","gold_delta","xp_delta"])
)


In [35]:
key_types = ["CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL","HERALD_KILL","DRAGON_KILL","BARON_KILL"]
events_by_min = (
    events_all
    .filter(pl.col("type").is_in(key_types))
    .group_by(["match_id","event_minute","type"])
    .len().rename({"len":"count"})
)
events_by_min

In [36]:
events_by_min

match_id,event_minute,type,count
str,i64,str,u32
"""EUW1_7460431991""",22,"""CHAMPION_KILL""",4
"""EUW1_7553544945""",8,"""CHAMPION_KILL""",1
"""EUW1_7460663774""",5,"""CHAMPION_KILL""",6
"""EUW1_7542824880""",42,"""ELITE_MONSTER_KILL""",1
"""EUW1_7553493341""",21,"""ELITE_MONSTER_KILL""",1
…,…,…,…
"""EUW1_7497313618""",16,"""CHAMPION_KILL""",1
"""EUW1_7527762593""",10,"""CHAMPION_KILL""",1
"""EUW1_7542946140""",5,"""CHAMPION_KILL""",5
